## Implementação da Decision Tree


In [1]:
import numpy as np
import pandas as pd
import math
df = pd.read_csv('tenis.csv')
df

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


## Entropia e Montagem da Arvore

In [2]:
def uncertainty(df, given):
    return entropy(df, None) - entropy(df, given)

In [3]:
def entropy(df, given):
    result = 0
    label = df.columns[len(df.columns)-1]
    distinct_label_values = df[label].unique()
    if given == None:
        for value in distinct_label_values:
            prob = df[df[label] == value].shape[0]/df.shape[0]
            x = df[df[label] == value].shape[0]/df.shape[0]
            log2 = 0 if x == 0 else math.log(x, 2)
            result += -prob*log2
    else:
        distinct_values = [distinct_label_values, df[given].unique()]
        for info_label in distinct_values[0]:
            for info_given in distinct_values[1]:            
              prob = df[(df[label] == info_label) & (df[given] == info_given)].shape[0]/df.shape[0]
              x = df[(df[label] == info_label) & (df[given] == info_given)].shape[0]/df[df[given] == info_given].shape[0]
              log2 = 0 if x == 0 else math.log(x, 2)
              result += -prob*log2
    return result

In [4]:
def decision_tree(df):
    return ['root', decision_tree_recursive(df, None)]

In [5]:
def decision_tree_recursive(df, parent):
    df = df if parent is None else df.drop([parent], axis=1)
    lowest_uncertainty = 0
    next_child = None
    for attr in df.columns[0:len(df.columns)-1]:
        u = uncertainty(df, attr)
        if u > lowest_uncertainty:
            lowest_uncertainty = u
            next_child = attr
    if lowest_uncertainty == 0:
        return df.iloc[0,len(df.columns)-1]
    child_values = df[next_child].unique()
    return [next_child, list(map(lambda c: [c, decision_tree_recursive(df[df[next_child] == c], next_child)], child_values))]

## Imprimir arvore

In [6]:
dec_test = decision_tree(df)

### Formato:
Dados de Array dentro de Array

In [7]:
dec_test

['root',
 ['outlook',
  [['sunny', ['humidity', [['high', 'no'], ['normal', 'yes']]]],
   ['overcast', 'yes'],
   ['rainy', ['windy', [[False, 'yes'], [True, 'no']]]]]]]